___

<a href='http://www.pieriandata.com'> <img src='../Pierian_Data_Logo.png' /></a>
___
# Text Generation with Neural Networks

In this notebook we will create a network that can generate text, here we show it being done character by character. Very awesome write up on this here: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

We organized the process into "steps" so you can easily follow along with your own data sets.

In [ ]:
# GOOGLE COLLAB USERS ONLY
%tensorflow_version 2.x

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
# IGNORE THE CONTENT OF THIS CELL
# tf.compat.v1.disable_eager_execution()
#import main libraries, import text and understand characters in text files
#import tensorflow as tf, rarely done but here needed, grab the data,

In [ ]:
tf.__version__

## Step 1: The Data

You can grab any free text you want from here: https://www.gutenberg.org/

We'll choose all of shakespeare's works (which we have already downloaded for you), mainly for two reasons:

1. Its a large corpus of text, its usually recommended you have at least a source of 1 million characters total to get realistic text generation.

2. It has a very distinctive style. Since the text data uses old style english and is formatted in the style of a stage play, it will be very obvious to us if the model is able to reproduce similar results.

2 main reasons, large work of text, >1 mil characters, and more importantly Shakespeare has distinct style and play, names are capitalized, someone enters or exits, undertsand 
create path to file, where text file is, notebook in same location as the file
read in with open(path_file, 'r').read() = read only mode.
Has spaces, \n, which stands for new line.

In [ ]:
path_to_file = 'shakespeare.txt'

In [ ]:
text = open(path_to_file, 'r').read()

In [ ]:
print(text[:500])

In [ ]:
print(text(4500:4800))

In [ ]:
print(text(140500:141500))

Clearly, distinct spaces here, new line, some number for a sonet and another sonet. network will be able to learn this.
Later will have plays, Helena speaks, expand., 500 more characters, King etc.
Well structured text.
Understand unique characters.

### Understanding unique characters

Get uniqye characters with sorted(set(text)), assign to vocab, new line, space, !, some numbers, capital and lowercase letters, punctuation ad
braces. 84 unique characters, keep in mind when designing last Dense layer.
Make sure to import tensorflow as tf and use correct file path.

In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print(vocab)
len(vocab)

## Step 2: Text Processing

### Text Vectorization

We know a neural network can't take in the raw string data, we need to assign numbers to each character. Let's create two dictionaries that can go from numeric index to character and character to numeric index.

In part 2, text processing, vectorising actual data, making dictionaries that network can go back and forth between, and also creating batches.
Previously we went ahead and created a set of all characters, sorted list.
Want to assign a number to each character.

In [ ]:
for pair in enumerate(vocab):
    print(pair)

Created loop, assigned number to every character, tuples.
Use this logic to generate a dictionary, that goes from character to certain index code.
Use character to index = {char:ind for ind, char in enumarate(vocab) - created dictionary, all charatcers are now the keys for index values.}

In [ ]:
char_to_ind = {u:i for i, u in enumerate(vocab)}

In [ ]:
char_to_ind

In [ ]:
ind_to_char = np.array(vocab)

In [ ]:
ind_to_char

In [ ]:
char_to_ind['H'] #returs 33

In [ ]:
ind_to_char = np.array(vocab)

Reverse, a vocab is a simple index call on the array, so use ind_to_char[33] get H. 
2 objects that easily to between characters, for all the text, encode it to integers.
Create encoded ytext with np.array([char_to_ind[c] for c in text]) - read text before, have entire text, encoded all of the text as an array.
Will take some time.

In [ ]:
encoded_text = np.array([char_to_ind[c] for c in text])

In [ ]:
encoded_text

In [ ]:
encoded_text.shape

Goes from 0 ... to 39, check shape, over 5 mill characters, that is enough for good results.

We now have a mapping we can use to go back and forth from characters to numerics.

If you had sample text of 20 indexes, first item in the sonnet, look at its encoded form, see all digits related to it.
Go from char to index encoding and from index to character.

In [ ]:
sample = text[:20]
sample

In [ ]:
encoded_text[:20]

## Step 3: Creating Batches

Overall what we are trying to achieve is to have the model predict the next highest probability character given a historical sequence of characters. Its up to us (the user) to choose how long that historic sequence. Too short a sequence and we don't have enough information (e.g. given the letter "a" , what is the next character) , too long a sequence and training will take too long and most likely overfit to sequence characters that are irrelevant to characters farther out. While there is no correct sequence length choice, you should consider the text itself, how long normal phrases are in it, and a reasonable idea of what characters/words are relevant to each other.

In step 3 we create the batchers, how text seq are organised and shift them 1 character/step forward.
Understand how Tensorflow generates batches use built in dataset class object to generate batches, and shuffle those batches.
We discovered how to take string data and encode it to have string assigned to it, and get bactches to train model on.
How long should sequence be?
Grab first 500 characters, general structure of the sonnet.

In [ ]:
print(text[:500])

Save single line

In [ ]:
line = "From fairest creatures we desire increase"

How long is that line? 41 characters, make sure training sequences are long enough so that model learns general patterns.
Shakespeare has specific structure due to the iambic pentameter, things rhyme to every other line, more or less.
Increase and decease, die and memory etc. Should have at least 3 lines to understand that structure, lines = ''' - triple quotes so multiline
41*3 = 133, choose seq length of 120 characters, smaller set of lines, seq length will depend on the structure of the text and what you want to
achieve.

In [ ]:
len(line)

In [ ]:
part_stanza = """From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,"""

In [ ]:
len(part_stanza)

### Training Sequences

The actual text data will be the text sequence shifted one character forward. For example:

Sequence In: "Hello my nam"
Sequence Out: "ello my name"


We can use the `tf.data.Dataset.from_tensor_slices` function to convert a text vector into a stream of character indices.

Choose seq length of 120, calculate total number of sequeces in text len(text)/seq_len, use // classic true division, round off
(seq_len+1), due to index 0, 45K sequences

In [ ]:
seq_len = 120

In [ ]:
total_num_seq = len(text)//(seq_len+1)

In [ ]:
total_num_seq

Use tensorflow object, data.Dataset.from_tesor_slices(on encoded text not the original text)
Will run for a both, special tensor sliced dataset,. char_dataset. use tab, manny commands, use batch method, converts sequences into batches
that can be fed onto the model for training.
What doess that look like?
for item in char_dataset.take - method, creates dataset with at most count elements, say 500, print items. convert to numpy()
see encoded text, use ind_to_char, printeed character by character, inside that large character dataset, create sequences from it.

In [ ]:
# Create Training Sequences
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

for i in char_dataset.take(500):
     print(ind_to_char[i.numpy()])

The **batch** method converts these individual character calls into sequences we can feed in as a batch. We use seq_len+1 because of zero indexing. Here is what drop_remainder means:

drop_remainder: (Optional.) A `tf.bool` scalar `tf.Tensor`, representing
    whether the last batch should be dropped in the case it has fewer than
    `batch_size` elements; the default behavior is not to drop the smaller
    batch.


Now create sequences in batches, use the batch method on char_dataset, this batch combines consecutive sequences into a batches, that can be fed
onto the network. Define first item as seq_len + 1, due to 0 index and drop remainder = True.
We did classic division, use true division, get decimals, not perfect fit, make sense, not expected perfect fit, characters at the end.
Drip remainder, drops last characters, 119 characters out of 5 million characters is not big deal.
Get sequences.

In [ ]:
sequences = char_dataset.batch(seq_len+1, drop_remainder=True)

Now that we have our sequences, we will perform the following steps for each one to create our target text sequences:

1. Grab the input text sequence
2. Assign the target text sequence as the input text sequence shifted by one step forward
3. Group them together as a tuple

Perform following steps to grab target sequences.
Grab seq, assign target seq and shift one char forwarsd, group together as tuple.
Create a function:
def create_seq_targets(seq):
    input_txt = seq[:-1] - from beginning except last char
    target_txt = seq[1:] - shifted 1 char forward
    return input_txt, target_txt

from hello my nam to ello my name, shifted by 1 char step into the future.

In [ ]:
def create_seq_targets(seq):
    input_txt = seq[:-1]
    target_txt = seq[1:]
    return input_txt, target_txt

Map to all sequences, map across the entire dataset, sequences.map(create_seq_targets)

In [ ]:
dataset = sequences.map(create_seq_targets)

For every pair of input and target seqs, print input text as numpy array, 
print index of characters, join that, print new line for clarity and print target text as numpy.
Print and join that with index to char, for target text.

Shows the first batch, 0 111 to 76 75, first sequences, what it looks like as characters below, network sees numbers only.
Then next sequence from 111 to 76 75 1, where 1 is white space, extrac white space, instead of T. 

In [ ]:
for input_txt, target_txt in  dataset.take(1):
    print(input_txt.numpy())
    print(''.join(ind_to_char[input_txt.numpy()]))
    print('\n')
    print(target_txt.numpy())
    # There is an extra whitespace!
    print(''.join(ind_to_char[target_txt.numpy()]))

### Generating training batches

Now that we have the actual sequences, we will create the batches, we want to shuffle these sequences into a random order, so the model doesn't overfit to any section of the text, but can instead generate characters given any seed text.

Now that we have sequences, generate batches, choose 128 batches, make sure to shuffle, so modern does not learn particular ordering of text,
but get random text and learn from it/
dataset = dataset.shuffle() to avoid shuffling entire text in memory use buffer_size = 10K, take 10K elements and shuffle them around, if
you don't specify buffer size, wil shuffle entire dataset and cause error, will take more memory, drop remainder = True.
Now should have dataset ready to go.
Different types of sequences, training and targe, both sizes of tuple (128, 128).
One of the most complex steps, 1 seq of encoded numbers that relates to a string, target is that sequence shifted by 1 character, more complex.
Text must be encoded as numbers and have index to char and char to index. Seq is 128 character long, input and target sequences.

In [ ]:
# Batch size
batch_size = 128

# Buffer size to shuffle the dataset so it doesn't attempt to shuffle
# the entire sequence in memory. Instead, it maintains a buffer in which it shuffles elements
buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

In [ ]:
dataset

## Step 4: Creating the Model

We will use an LSTM based model with a few extra features, including an embedding layer to start off with and **two** LSTM layers. We based this model architecture off the [DeepMoji](https://deepmoji.mit.edu/) and the original source code can be found [here](https://github.com/bfelbo/DeepMoji).

The embedding layer will serve as the input layer, which essentially creates a lookup table that maps the numbers indices of each character to a vector with "embedding dim" number of dimensions. As you can imagine, the larger this embedding size, the more complex the training. This is similar to the idea behind word2vec, where words are mapped to some n-dimensional space. Embedding before feeding straight into the LSTM usually leads to more realisitic results.

Creating the model by setting up loss function, simple model with 3 layers, witj 1 neuron per vocab character. 
vocab_size is length of vocab or unique characters, 84
embedding dimension for embedding layer, choose 64 as same range as vocab_size, not too large, as some features discovered then won't be useful
single layer with rnn_neurons, many , 1026 or 1024, single layer with a lot of neurons
choose multiple layers, LSTM and GRU stacked on top of each other, choose simple model that can produce realistic results

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embed_dim = 64

# Number of RNN units
rnn_neurons = 1026

Now let's create a function that easily adapts to different variables as shown above.

import all layers, with LSTM, Dense, Embedding, Dropout and Gru
As labels are one hot encoded, use sparse categorical crossentropy, instead of categorical crossentropy.
link, check it out, also use help on it, from logits = False as one hot encoded.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding,Dropout,GRU

### Setting up Loss Function

For our loss we will use sparse categorical crossentropy, which we can import from Keras. We will also set this as logits=True

In [ ]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [ ]:
help(sparse_categorical_crossentropy)

https://datascience.stackexchange.com/questions/41921/sparse-categorical-crossentropy-vs-categorical-crossentropy-keras-accuracy

Wrap it, taking y_true and y_pred, just as the original does, return sparse categorical function(y_true, y_pred, logits=T)
Create custom loss function, specify the parameter to be True as default is false.

In [ ]:
def sparse_cat_loss(y_true,y_pred):
  return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

Save model in function, as you change dataset, you can easily adapt your model.
Function takes in vocab size, embedding  dimensions, number of neurons RNN andd batch size.
model starts witj Sequential

add layers 1st is Embedding, embedds into some amount of dimensions, pass vocab size, input_dimensions, embedding dimensions as output_dimensions
then batch input shape **kwargs argument, batch_size by None

add GRU layer, with rnn_neuron number, can expand, a couple of parameters, can specify droput call within, set return_sequebces = True and stateful = True, we need to return last output in output seq or the full sequence, we are dealing with seqs shifted over, see history
stateful = True, last state of each sample at index i in a batch will be used as initial state for the sample index i in the following batch, keep current state.
recurrent_initializer = actual initializer of weight values for RNN, set to, several string codes, default is glorot uniform and recurrent as orthogonal, but recurrent performs better as glorot uniform too, name of person who developed it, typically named last name, he still works in DeepMind

Finally add Dense layer, just the vocab size

Compile model with adam optimizer and our loss function, pass variable than call the function, had to edit the function to specify logist=True,
cannot do that by calling, will be default setting instead that is False.

return the model

In [ ]:
def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim,batch_input_shape=[batch_size, None]))
    model.add(GRU(rnn_neurons,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
    # Final Dense Layer to Predict
    model.add(Dense(vocab_size))
    model.compile(optimizer='adam', loss=sparse_cat_loss) 
    return model

Use function to create the model, use your variables.

In [ ]:
model = create_model(
  vocab_size = vocab_size,
  embed_dim=embed_dim,
  rnn_neurons=rnn_neurons,
  batch_size=batch_size)

View the model, see your settings.
Shapes are 128, None, 64 - batch_size, later will be changed when model is reloaded, and batch size will be 1.
Check model, without any training to get random characters to check it works.

In [ ]:
model.summary()

## Step 5: Training the model

Let's make sure everything is ok with our model before we spend too much time training! Let's pass in a batch to confirm the model currently predicts random characters without any training.



Training model, show example of how to train the model, over 3 mill parameters, requires proper GPU and time, show how to load the model.
Also show how to check if model works correctly, run model with no training, except random characters.
Whenn calling model summary, 3 mill parameters, need a proper GPU and time to train, requires Kuda and GPU.
Use Google colab, upload notebook, file - upload notebook, then choose file.
Upload textdata - expand, files, make sure it is connnected to runtime, upload, shakespeare, or upload both model and text
Text and trained model, load up later on, make sure to use tensorflow 2 so %tensorflow_version 2.x, hit runtime and run all, set TF as 2.0
once ready reads in file and does encoding and decoding, and see model summary.
>3 mil parameters, make sure to change GPU.

Check model is okay
Use 1 batch for this, example simple batch and pass into the mdoel
for input_example_batch, targe_example_bacth in dataset.take 1:
    example batch predictions = model(input_example_batch)
example batch = original target batch is shifted +1
Should predict 1 into the future

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):

  # Predict off some random batch
  example_batch_predictions = model(input_example_batch)

  # Display the dimensions of the predictions
  print(example_batch_predictions.shape, " <=== (batch_size, sequence_length, vocab_size)")


In [ ]:
example_batch_predictions.shape

Check, 128 is batch size, 120 is seq lence of characters, 84 is vocab size
See first 0, bunch of probabilities for each character.
Use tf commands, tf.random.categorical, takes logartic probability, num_samples = 1.
Want single sample, sampled_indices, get probability distribution, sampled_indices, check get ind to character, indices of predicted characters.

In [ ]:
example_batch_predictions

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)

In [ ]:
sampled_indices

Use tf squeeze, pass in sampled indices, axis = -1 , numpy() to reshape it, format we want it in.
Then pass in ind_to_char dictionary index, get bunch of random characters, get prediction, random, model hasn't be trained.

In [ ]:
# Reformat to not be a lists of lists
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [ ]:
sampled_indices

In [ ]:
print("Given the input seq: \n")
print("".join(ind_to_char[input_example_batch[0]]))
print('\n')
print("Next Char Predictions: \n")
print("".join(ind_to_char[sampled_indices ]))


After confirming the dimensions are working, let's train our network!

Choose number of epochs, must be >=30 to get realistic and reasonable results.
Do model.fit(dataset object, epochs= epochs)
Fit the model, will take a long time to run, 1 - 2 min per epoch, especially for such a large mode, runtime tab, interrupt execution.
Stop running the model, or you can wait.
Load up the model instead.

In [ ]:
epochs = 30

In [ ]:
model.fit(dataset,epochs=epochs)

## Step 6: Generating text

Currently our model only expects 128 sequences at a time. We can create a new model that only expects a batch_size=1. We can create a new model with this batch size, then load our saved models weights. Then call .build() on the model:

In [ ]:
model.save('shakespeare_gen.h5') 

from tensorflow.keras.models import load_model, for Colab you must upload the model.
Load the model weights and built another batch set.

In [ ]:
from tensorflow.keras.models import load_model

Final part, load up the model, load the weights and accept batch size of 1.
Then create function to build generated text.

use function create_model, pass in everything, batch_size = 1, create model with 1 batch size
model.load_weights('previous_model.h5') - load from previous model
Then say model.build(tf.TensorShape(1, None))
Run 3 lines, create new instance of model, and instead of loading up full model, just load the weights with that shape size.
Created model with 1 batch size.

In [ ]:
model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)

model.load_weights('shakespeare_gen.h5')

model.build(tf.TensorShape([1, None]))


In [ ]:
model.summary()

Create function to generate text
Takes in model, start seed, gen_size, also temperature setting, set to 1.0 by default.
Basic idea is to take some seed text, formatt it for the right shape for the network, loop the sequence as we keep adding our own predicted characters.
Similar to time series appending predictions.
num_gen = gen_size, num_characters to generate
start.seed is raw string
inp_evaluation = [char_to_und[s], for s in start_seed] - convert each character to index
We now have to match the batch shape, so input_eval = tf.expand_dims(input_eval, 0) - expands dims and ensures correct text
text_generated = [] to hold generated text
temperature = temp. can make it lower or higher, so based on random or expected you want the text to be
reset model state
model.reset_state()
for i in range(num_gen):
predictions = model(input_eval)
remove batch sha[e predictions = tf.squeeze(predictions, 0]
predictions = predictions/temperature
predicted_ind = tf.random.categorical(predictions, num_samples = 1) just need 1 prediction, index formatting so use [-1,0].numpy()
input_eval = tf.expand_dims([predicted_ind])
and transform back to characters text_generated.append(ind_to_char[predicted_ind])

Check indentation and brackets, returns start seed and join all items
return (start_seed + ''.join(text_generated))

In [ ]:
def generate_text(model, start_seed,gen_size=100,temp=1.0):
  '''
  model: Trained Model to Generate Text
  start_seed: Intial Seed text in string form
  gen_size: Number of characters to generate

  Basic idea behind this function is to take in some seed text, format it so
  that it is in the correct shape for our network, then loop the sequence as
  we keep adding our own predicted characters. Similar to our work in the RNN
  time series problems.
  '''

  # Number of characters to generate
  num_generate = gen_size

  # Vecotrizing starting seed text
  input_eval = [char_to_ind[s] for s in start_seed]

  # Expand to match batch format shape
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty list to hold resulting generated text
  text_generated = []

  # Temperature effects randomness in our resulting text
  # The term is derived from entropy/thermodynamics.
  # The temperature is used to effect probability of next characters.
  # Higher probability == lesss surprising/ more expected
  # Lower temperature == more surprising / less expected
 
  temperature = temp

  # Here batch size == 1
  model.reset_states()

  for i in range(num_generate):

      # Generate Predictions
      predictions = model(input_eval)

      # Remove the batch shape dimension
      predictions = tf.squeeze(predictions, 0)

      # Use a cateogircal disitribution to select the next character
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # Pass the predicted charracter for the next input
      input_eval = tf.expand_dims([predicted_id], 0)

      # Transform back to character letter
      text_generated.append(ind_to_char[predicted_id])

  return (start_seed + ''.join(text_generated))

Take in model, start seed string, num characters to generate and temperature, get input evaluation, transform string back to index locations.
transform shape for the batch, reset state of the model, generate characters, squeeze predictions to undo dimensions to grab predictions, divide by temperature to randomize or not. or /1 is leaving the same

get predicted id, categorical, convert to numpy, get digits, input eval, shifted by 1,  append to predictions,convert to characters, return characters


Print, run function, pass flower, "JULIET", gen_size=1000 characters)), takes a while, see Shakespearan text.
See Julliet, King Richard and other characters.
Some words are madeup but some words make sense. Some typos, but character by character basis, see labelling of scenes, network learned structure.
Learned characters as all CAPS and what they say, dialogue.
Able to learn Shakespeare structure.
Check link for more examples of RNN that generates results.

In [ ]:
print(generate_text(model,"flower",gen_size=1000))

In [ ]:
print(generate_text(model, "JULIET", gen_size=1000))